In [47]:
import pandas as pd
import numpy as np
import math
import geopandas as gpd
from shapely.geometry import Point

import sys
sys.path.append("/Users/Christine/MSAN_module4/msan631/SFpal/functions/data_transforming")
from spatial_analysis import *

# Clip zip code shapefile to only SF

In [40]:
communities = gpd.read_file('../data/raw_data/zip_codes/geo_export_37348335-ac06-4ade-bf6d-453d393d539d.shp')

In [41]:
communities = communities[communities["po_name"] == "SAN FRANCISCO"]

In [42]:
communities.to_file('../data/clean_data/SF_zip_codes/SF_zip_codes.shp')

# Clean/Process parks data

In [43]:
parks = pd.read_csv('../data/raw_data/parks_data.csv')

In [44]:
parks = parks.iloc[1:,:]
parks = parks.dropna(subset = ["Location 1"])

In [45]:
temp = parks["Location 1"].apply(lambda x: x.split('\n'))
parks["address"] = temp.apply(lambda x: x[0])
parks["city_state"] = temp.apply(lambda x: x[1])
parks["lat_long"] = temp.apply(lambda x: x[2])
del parks["Location 1"]
del parks["Lat"]
del parks["Zipcode"]

In [46]:
# Add zip code field to PARKS data using spatial join
parks = spatial_join_zipcode(parks, communities)

In [17]:
parks.to_csv('../data/clean_data/parks.csv', index=False)

# Clean/Process schools data

In [34]:
schools = pd.read_csv('../data/raw_data/Schools.csv')

In [35]:
schools = schools.dropna(subset = ["Location 1"])
temp = schools["Location 1"].apply(lambda x: x.split('\n'))
schools["State"] = temp.apply(lambda x: x[0])
schools["lat_long"] = temp.apply(lambda x: x[1])
del schools["Location 1"]
del schools["State"]

In [36]:
# Add zip code field to SCHOOLS data using spatial join
schools = spatial_join_zipcode(schools, communities)

In [83]:
schools.to_csv('../data/clean_data/schools.csv', index=False)

# Clean/Process bart stations data

In [37]:
bart_stations = gpd.read_file('../data/raw_data/bart_stations/BART_Sta_13.shp')

In [85]:
# Add zip code field to BART STATIONS data using spatial join
bart_stations = bart_stations.to_crs({'init': 'epsg:4326'})
bart_stations = gpd.sjoin(bart_stations, communities, how="inner", op='intersects')
del bart_stations["index_right"]

In [87]:
bart_stations.to_csv('../data/clean_data/bart_stations.csv', index=False)